# Livestock Market Analysis

Analysis of the Livestock Market in the Caribean Region of Colombia.

### OCR With Tesseract

OCR part done with Tesseract to recognize the characters in the pdf's.

In [43]:
# Import modules for OCR
try:
    import Image
except ImportError:
    from PIL import Image
import pytesseract

# Import os for getting image filenames in img directory
from os import listdir

In [84]:
# The cutoff represents the file where a new format starts
cutoff = "14-01-07.jpg"
# Livestock lot types
livestock_types = ["HL", "HV", "VE", "VP", "ML", "MC", "TO", "BH", "BM"]
# Categories of iron for female/male
iron_categories = ["HEMBRA DE 1a", "HEMBRA DE 2a", "MACHO DE 1a", "MACHO DE 2a"]
# Digits array
digits = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

In [45]:
# Image filenames 
image_filenames = [f for f in listdir("img") if ".jpg" in f]

In [46]:
# Auction filenames list
auction_filenames = []

# Temp ary of auction filename
temp_ary = []

# Generate auction filenames list
for index, filename in enumerate(image_filenames):
    # Append filename to temp_ary
    temp_ary.append(filename)
    # If it is the end append temp_ary no matter what
    if index == len(image_filenames)-1:
        auction_filenames.append(temp_ary)
    else:
        # Select next filename root to check if it belongs to the same multipage file
        next_root_filename = "-".join(image_filenames[index+1].replace(".jpg", "").split("-")[0:3])
        # If the next filename is another page of the same file go on and keep appending to temp_ary
        if next_root_filename in filename:
            pass
        # Else apppend temp ary and empty it
        else:
            auction_filenames.append(temp_ary)
            temp_ary = []

In [47]:
# Check if it coincides with the number of pdf's records gotten from the auction webstie
if len(auction_filenames) == 458:
    print("Ente gut, alles gut!")

Ente gut, alles gut!


In [83]:
# Count to limit number of files for testing
count = -1

# Loop through each auction and process the image data into the dataframe
for auction_ary in auction_filenames:
    # Break on the count file iteration
    count += 1
    if count == 1:
        break
    # Category of livestock: Number of iron brands: 1st male, 2nd male, 1st female, 2nd female.
    current_category = ""
    # Auction filename
    for auction_filename in auction_ary:
        current_category = append_image_to_dataframe(auction_filename, current_category)

img/13-01-08.jpg
['2 3/4', 'HV', '5', '370', '2,580', '2,580', '2,580', '954,600']
['XX', 'VE', '76', '472', '2,380', '2,740', '2,557', '1,210,308']
['XX', 'BH', '1', '404', '1,000', '1,000', '1,000', '404,000']
['XX', 'TO', '8', '647', '2,680', '2,860', '2,765', '1,785,980']
['2', 'HV', '1', '268', '1,800', '1,800', '1,800', '482,400']
['2 1/4', 'HV', '1', '298', '2,200', '2,200', '2,200', '655,600']
['2 1/2', 'HV', '10', '345', '2,340', '2,400', '2,373', '816,368']
['2 3/4', 'HV', '1', '364', '2,600', '2,600', '2,600', '946,400']
['XX', 'VE', '163', '391', '1,950', '2,560', '2,279', '903,677']
['2 1/2', 'MC', '1', '342', '2,550', '2,550', '2,550', '872,100']
['2 3/4', 'MC', '1', '396', '2,640', '2,640', '2,640', '1,045,440']
['XX', 'TO', '3', '507', '2,740', '2,780', '2,760', '1,399,293']
################################################


In [51]:
# Method that pulls data from image and append it to dataframe. It returns the last iron brand category 
def append_image_to_dataframe(auction_filename, current_category):
    img_path = "img/{0}".format(auction_filename)
    print(img_path)
    img = Image.open(img_path)
    text = pytesseract.image_to_string(img)
    text_lines = text.split("\n")
    for line in text_lines:
        output = is_data_entry(line)
        if output[0] is True: print(output[1])
    print("################################################")

In [85]:
# Return if the line corresponds to the category of an iron. If so, return the name of the category
def is_iron_category(line):
    # Check if there is an iron category in the line
    has_iron_category = [category in line for category in iron_categories]
    # Business logic
    if True in has_iron_category:
        return True, iron_categories[has_iron_category.index(True)] 
    return False, None

In [86]:
# Return if the line corresponds to a data entry. If so, return the line 
def is_data_entry(line):
    # Number of digits in the line
    number_of_digits = sum([line.count(digit) for digit in digits])
    # Bussiness logic
    has_livestock_type = [livestock_type in line for livestock_type in livestock_types]
    if (number_of_digits > 10) and (True in has_livestock_type):
        # Return True and the data line with the corrections of erros introduced by OCR routine
        # and properly formatted into list, ready to enter it into data frame 
        return True, formatted_data_line(line, livestock_types[has_livestock_type.index(True)])
    return False, None

In [87]:
# Correct common errors in data line that come out of OCR routine
def formatted_data_line(line, livestock_type):
    # Replace " ," by ",". Some numbers appear broken
    line = line.replace(" ,", ",")
    # Remove double spaces if any
    while "  " in line:
        line.replace("  ", " ")
    # Break up data line into columns
    age, numerical_data = line.split(" {0} ".format(livestock_type))
    return [age, livestock_type, *numerical_data.split(" ")]